# Objecs in Boxes
___

Problem: Put N objects which are scattered in the plane, into a row of N boxes.
Constrains:
- Boxes are aligned from left to right (if i<i′, box i is to the left of box i′) on the x axis.
- Box i is located at a point Bi of the (x,y) plane and object j is located at Oj.
- We want to find an arrangement of objects such that:
- each box contains exactly one object,
- each object is stored in one box, 
- the total distance from object j  to its storage box is minimal.

Step 1: Import the library

In [1]:
import sys
try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/') 

In [2]:
try:
    import cplex
except:
    raise Exception('Please install CPLEX. See https://pypi.org/project/cplex/')

Step 2: Model the data

The input data is the number of objects (and boxes) N, and their positions in the (x,y) plane.

Step 3: Prepare the data

We use Euclidean distance to compute the distance between an object and its assigned box.

In [3]:
from math import sqrt

N = 15
box_range = range(1, N+1)
obj_range = range(1, N+1)

import random

o_xmax = N*10
o_ymax = 2*N
box_coords = {b: (10*b, 1) for b in box_range}

obj_coords= {1: (140, 6), 2: (146, 8), 3: (132, 14), 4: (53, 28), 
             5: (146, 4), 6: (137, 13), 7: (95, 12), 8: (68, 9), 9: (102, 18), 
             10: (116, 8), 11: (19, 29), 12: (89, 15), 13: (141, 4), 14: (29, 4), 15: (4, 28)}

# the distance matrix from box i to object j
# actually we compute the square of distance to keep integer
# this does not change the essence of the problem
distances = {}
for o in obj_range:
    for b in box_range:
        dx = obj_coords[o][0]-box_coords[b][0]
        dy = obj_coords[o][1]-box_coords[b][1]
        d2 = dx*dx + dy*dy
        distances[b, o] = d2

Step 4: Set up the prescriptive model

In [4]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Windows 64bit
* Python version 3.10.5, located at: c:\Users\gviac\AppData\Local\Programs\Python\Python310\python.exe
* docplex is present, version is 2.24.232
* CPLEX library is present, version is 22.1.0.0, located at: c:\Users\gviac\AppData\Local\Programs\Python\Python310\lib\site-packages
* pandas is present, version is 1.4.3


Create the DOcplex model

The model contains all the business constraints and defines the objective.

In [5]:

from docplex.mp.model import Model

mdl = Model(name="boxes")

Define the decision variables

- For each box i (i in 1..N) and object j (j in 1..N), we define a binary variable Xi,j equal to 1 if and only if object j is stored in box i.

Note that the name parameter is actually a function, this function takes a key pair ij  and coins a new name for each corresponding variables. The name parameter also accepts a string prefix, in which case, Docplex will generate names by concatenating the prefix with the string representation of keys.

In [6]:
# decision variables is a 2d-matrix
x = mdl.binary_var_matrix(box_range, obj_range, name=lambda ij: "x_%d_%d" %(ij[0], ij[1]))

Express the business constraints
- The sum of  over both rows and columns must be equal to , resulting in  constraints.

In [7]:

# one object per box
mdl.add_constraints(mdl.sum(x[i,j] for j in obj_range) == 1 for i in box_range)
    
# one box for each object
mdl.add_constraints(mdl.sum(x[i,j] for i in box_range) == 1 for j in obj_range)

# model information
mdl.print_information()

Model: boxes
 - number of variables: 225
   - binary=225, integer=0, continuous=0
 - number of constraints: 30
   - linear=30
 - parameters: defaults
 - objective: none
 - problem type is: MILP


Express the objective
- The objective is to minimize the total distance between each object and its storage box.

In [8]:
# minimize total displacement
mdl.minimize(mdl.dotf(x, lambda ij: distances[ij]))

Solve the model

In [9]:
mdl.print_information()

assert mdl.solve(log_output=True), "!!! Solve of the model fails"

Model: boxes
 - number of variables: 225
   - binary=225, integer=0, continuous=0
 - number of constraints: 30
   - linear=30
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 106038.000000 after 0.00 sec. (0.02 ticks)
Found incumbent of value 23518.000000 after 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
Reduced MIP has 30 rows, 225 columns, and 450 nonzeros.
Reduced MIP has 225 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.26 ticks)
Probing time = 0.00 sec. (0.35 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 30 rows, 225 columns, and 450 nonzeros.
Reduced MIP has 225 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.30 ticks)
Probing time = 0.00 sec. (0.35 ticks)
Clique table members: 30.
MIP emphasis: balance optimality and feasibility.
MIP search

In [10]:
mdl.report()
d1 = mdl.objective_value
#mdl.print_solution()

def make_solution_vector(x_vars):
    sol = [0]* N
    for i in box_range:
        for j in obj_range:
            if x[i,j].solution_value >= 0.5:
                sol[i-1] = j
                break
    return sol

def make_obj_box_dir(sol_vec):
    # sol_vec contains an array of objects in box order at slot b-1 we have obj(b)
    return { sol_vec[b]: b+1 for b in range(N)}
    
               
sol1 = make_solution_vector(x)
print("* solution: {0!s}".format(sol1))  

* model boxes solved with objective = 8858.000
* solution: [15, 11, 14, 4, 8, 12, 7, 9, 10, 3, 6, 1, 13, 2, 5]


Additional constraint #1

Additional constraint #2